In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection as ms
from sklearn import preprocessing

%matplotlib inline


Load and join datasets. 

In [22]:
bikeStats = pd.read_csv('AdvWorksCusts.csv')
bikeBuyer = pd.read_csv('AW_BikeBuyer.csv')
bikes = pd.merge(bikeStats, bikeBuyer, on='CustomerID')

print(bikes.shape)
print(bikes.CustomerID.unique().shape)
print(bikes['BikeBuyer'].value_counts())
bikes.head()

(16749, 24)
(16404,)
0    11188
1     5561
Name: BikeBuyer, dtype: int64


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Bachelors,Professional,M,M,1,0,0,2,137947,0
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Bachelors,Professional,M,S,0,1,3,3,101141,1
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Bachelors,Professional,M,M,1,1,3,3,91945,0
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Bachelors,Professional,F,S,0,1,0,0,86688,0
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Bachelors,Professional,F,S,1,4,5,5,92771,1


In [23]:
bikes.dtypes

CustomerID               int64
Title                   object
FirstName               object
MiddleName              object
LastName                object
Suffix                  object
AddressLine1            object
AddressLine2            object
City                    object
StateProvinceName       object
CountryRegionName       object
PostalCode              object
PhoneNumber             object
BirthDate               object
Education               object
Occupation              object
Gender                  object
MaritalStatus           object
HomeOwnerFlag            int64
NumberCarsOwned          int64
NumberChildrenAtHome     int64
TotalChildren            int64
YearlyIncome             int64
BikeBuyer                int64
dtype: object

In [24]:
#find unique item values with counts
for col in bikes.columns:
    print(col, "\n", bikes[col].value_counts(), "\n\n")
    

CustomerID 
 17390    4
18175    4
11172    4
22811    4
18689    4
17494    4
11999    4
18719    4
21353    4
25948    4
18083    4
13139    4
14621    4
22774    4
17543    4
11143    4
11442    4
15512    4
24328    4
26358    4
15531    4
13289    4
18797    4
19315    4
26443    4
14477    4
12144    4
18487    4
26600    4
11430    4
        ..
17714    1
19763    1
28007    1
25958    1
19811    1
17762    1
23905    1
21856    1
11615    1
15709    1
13660    1
27991    1
25942    1
19795    1
17746    1
23889    1
21840    1
11599    1
15693    1
13644    1
27975    1
25926    1
19779    1
17730    1
23873    1
11583    1
15677    1
13628    1
25910    1
12294    1
Name: CustomerID, Length: 16404, dtype: int64 


Title 
 Mr.     43
Ms.     38
Sr.      4
Mrs.     2
Ms       1
Name: Title, dtype: int64 


FirstName 
 Isabella     91
Lucas        88
Eduardo      88
Marcus       87
Jennifer     87
Richard      86
Alexandra    85
Dalton       85
Katherine    85
Morgan       84
Kai

In [25]:
#search for empty cells
(bikes.astype(np.object) == '').any()

CustomerID              False
Title                   False
FirstName               False
MiddleName              False
LastName                False
Suffix                  False
AddressLine1            False
AddressLine2            False
City                    False
StateProvinceName       False
CountryRegionName       False
PostalCode              False
PhoneNumber             False
BirthDate               False
Education               False
Occupation              False
Gender                  False
MaritalStatus           False
HomeOwnerFlag           False
NumberCarsOwned         False
NumberChildrenAtHome    False
TotalChildren           False
YearlyIncome            False
BikeBuyer               False
dtype: bool

Feature Engeneering
Transforms BirthDate into Age in 2019.
Takes the AreaCode from the PhoneNumber.

In [26]:
Age = [2019 - int(x[0:4]) for x in bikes.BirthDate]
bikes['Age'] = Age

AreaCode = [int(x[7:10]) if '(' in x else int(x[4:7]) for x in bikes.PhoneNumber]
bikes['AreaCode'] = AreaCode

Sex = [1 if x == 'M' else 0 for x in bikes.Gender]
bikes['Sex'] = Sex

Married = [1 if x == 'M' else 0 for x in bikes.MaritalStatus]
bikes['Married'] = Married

jobs = {'Professional' : 0, 'Skilled Manual' : 1, 'Management' : 2, 'Clerical' : 3, 'Manual' : 4}
JobType = [jobs[x] for x in bikes.Occupation]
bikes['JobType'] = JobType

educ = {'Bachelors ' : 0, 'Partial College' : 1, 'High School' : 2, 'Graduate Degree' : 3, 'Partial High School' : 4}
EducationType = [educ[x] for x in bikes.Education]
bikes['EducationType'] = EducationType

country = {'United States' : 0, 'Australia' : 1, 'United Kingdom' : 2, 'France' : 3, 'Germany' : 4, 'Canada' : 5}
Country = [country[x] for x in bikes.CountryRegionName]
bikes['Country'] = Country

Income = []
for i in bikes['YearlyIncome']:
    if i <= 50000:
        Income.append(0)
    elif i <= 75000:
        Income.append(1)
    elif i <= 100000:
        Income.append(2)
    else:
        Income.append(3)
    #if i <= 30000:
    #    Income.append(0)
    #elif i <= 60000:
    #    Income.append(1)
    #elif i <= 90000:
    #    Income.append(2)
    #else:
    #    Income.append(3)
    
bikes['Income'] = Income 

#ChildrenAtHomeFlag = [0 if x == 0 else 1 for x in bikes.NumberChildrenAtHome]
#bikes['ChildrenAtHomeFlag'] = ChildrenAtHomeFlag

# min age 39 and max age 109
AgeBracket = []
for i in bikes.Age:
    if i < 54:
        AgeBracket.append(0)
    elif i < 69:
        AgeBracket.append(1)
    elif i < 84:
        AgeBracket.append(2)
    else:
        AgeBracket.append(3)
        
bikes['AgeBracket'] = AgeBracket

ChildrenFlag = [0 if x == 0 else 1 for x in bikes.TotalChildren]
bikes['ChildrenFlag'] = ChildrenFlag

#OnePlusCarsFlag = [0 if x <= 1 else 1 for x in bikes.NumberCarsOwned]
#bikes['OnePlusCarsFlag'] = OnePlusCarsFlag

Remove duplicate rows.

In [27]:
bikes.drop_duplicates(subset = 'CustomerID', keep = 'first', inplace = True)
print(bikes.shape)
print(bikes.CustomerID.unique().shape)
print(bikes['BikeBuyer'].value_counts())

(16404, 34)
(16404,)
0    10953
1     5451
Name: BikeBuyer, dtype: int64


In [28]:
labels = ['Country', 'EducationType', 'JobType', 'Sex', 'Married', 'HomeOwnerFlag', 'NumberCarsOwned', 
                            'NumberChildrenAtHome', 'ChildrenFlag', 'Income', 'AgeBracket', 'AreaCode', 'BikeBuyer']
for col in bikes.columns:
    if col not in labels:
        bikes.drop(col, axis=1, inplace=True)
#features = pd.get_dummies(bikes)
print(bikes.shape)
bikes.dtypes

(16404, 13)


HomeOwnerFlag           int64
NumberCarsOwned         int64
NumberChildrenAtHome    int64
BikeBuyer               int64
AreaCode                int64
Sex                     int64
Married                 int64
JobType                 int64
EducationType           int64
Country                 int64
Income                  int64
AgeBracket              int64
ChildrenFlag            int64
dtype: object

In [29]:
#find unique item values with counts
for col in bikes.columns:
    print(col, "\n", bikes[col].value_counts(), "\n\n")

HomeOwnerFlag 
 1    11058
0     5346
Name: HomeOwnerFlag, dtype: int64 


NumberCarsOwned 
 2    5741
1    4325
0    3782
3    1429
4    1127
Name: NumberCarsOwned, dtype: int64 


NumberChildrenAtHome 
 0    9924
1    2180
2    1455
3    1057
4     943
5     845
Name: NumberChildrenAtHome, dtype: int64 


BikeBuyer 
 0    10953
1     5451
Name: BikeBuyer, dtype: int64 


AreaCode 
 555    8398
500    8006
Name: AreaCode, dtype: int64 


Sex 
 1    8290
0    8114
Name: Sex, dtype: int64 


Married 
 1    8852
0    7552
Name: Married, dtype: int64 


JobType 
 0    4922
1    4038
2    2718
3    2597
4    2129
Name: JobType, dtype: int64 


EducationType 
 0    4758
1    4451
2    2944
3    2851
4    1400
Name: EducationType, dtype: int64 


Country 
 0    6935
1    3194
2    1709
3    1591
4    1577
5    1398
Name: Country, dtype: int64 


Income 
 3    4849
0    4495
1    3556
2    3504
Name: Income, dtype: int64 


AgeBracket 
 0    6812
1    6690
2    2636
3     266
Name: AgeBracket

In [30]:
bikes.to_csv('BikesProcessed.csv')
bikes.head()

,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,BikeBuyer,AreaCode,Sex,Married,JobType,EducationType,Country,Income,AgeBracket,ChildrenFlag
0,1,0,0,0,500,1,1,0,0,1,3,0,1
1,0,1,3,1,500,1,0,0,0,1,3,1,1
2,1,1,3,0,500,1,1,0,0,1,2,1,1
3,0,1,0,0,500,0,0,0,0,1,2,0,0
4,1,4,5,1,500,0,0,0,0,1,2,0,1


One hot encoding.

labels = ['CountryRegionName', 'Education', 'Occupation', 'Sex', 'Married', 'HomeOwnerFlag', 'NumberCarsOwned', 
                            'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'Age', 'AreaCode', 'BikeBuyer']
for col in bikes.columns:
    if col not in labels:
        bikes.drop(col, axis=1, inplace=True)
features = pd.get_dummies(bikes)
print(features.shape)
features.dtypes

features.to_csv('BikesProcessed.csv')
features.head()

Features that are not included in the dataset:
    'BirthDate', 'CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1', 'AddressLine2', 'PhoneNumber'
    
Code below is based on:
https://www.edx.org/course/principles-of-machine-learning-python-edition

def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['StateProvinceName', 'CountryRegionName', 'PostalCode', 'Education', 'Occupation', 
                       'Gender', 'MaritalStatus']

features = encode_string(bikes['City'])
for col in categorical_columns:
    temp = encode_string(bikes[col])
    features = np.concatenate([features, temp], axis = 1)
    
features = np.concatenate([features, np.array(bikes[['HomeOwnerFlag', 'NumberCarsOwned', 
                            'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'Age', 'AreaCode', 'BikeBuyer']])], axis = 1)
    
print(features.shape)
print(features[:2, :]) 

Save processed data as a new dataset.

processedData = pd.DataFrame(features)
processedData.to_csv('BikesProcessed.csv')
processedData.head()